In [1]:
import pandas as pd

In [2]:
!pip install -q datasets transformers[torch] accelerate jiwer tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 56.7 MB/s eta 0:00:00


In [3]:
!pip install -q librosa soundfile

In [4]:
from datasets import load_dataset
import IPython.display as ipd

In [5]:
ds = load_dataset("ysdede/khanacademy-turkish")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00003.parquet:   0%|          | 0.00/392M [00:00<?, ?B/s]

data/train-00001-of-00003.parquet:   0%|          | 0.00/398M [00:00<?, ?B/s]

data/train-00002-of-00003.parquet:   0%|          | 0.00/397M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25741 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1355 [00:00<?, ? examples/s]

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['transcription', 'audio'],
        num_rows: 25741
    })
    test: Dataset({
        features: ['transcription', 'audio'],
        num_rows: 1355
    })
})

In [9]:
f"\ntrain samples: {len(ds['train'])}"

'\ntrain samples: 25741'

In [10]:
ds['train'][0]

{'transcription': 'Bunu belirleyen iki dinamik var birincisi litre başına sabit maliyetimizi Asgariye düşürebilmek için mümkün olduğunca çok üretmek isteriz.',
 'audio': <datasets.features._torchcodec.AudioDecoder at 0x79439cfc4b90>}

In [11]:
train_subset = ds['train'].select(range(int(len(ds['train']) * 0.15)))
test_subset = ds['test'].select(range(int(len(ds['test']) * 0.15))) #vaxt az olduğu üçün 15%

In [12]:
f"train: {len(train_subset)}, test: {len(test_subset)}"

'train: 3861, test: 203'

In [13]:
sample = train_subset[0]

In [14]:
f"\ntext: {sample['transcription']}"

'\ntext: Bunu belirleyen iki dinamik var birincisi litre başına sabit maliyetimizi Asgariye düşürebilmek için mümkün olduğunca çok üretmek isteriz.'

In [15]:
f"sample rate: {sample['audio']['sampling_rate']}"

'sample rate: 16000'

In [17]:
f"audio length: {len(sample['audio']['array'])/ sample['audio']['sampling_rate']:.1f}s"

'audio length: 9.3s'

In [18]:
import numpy as np

In [19]:
check_idx = range(min(500, len(train_subset)))
durations = []
for i in check_idx:
    a = train_subset[i]['audio']
    durations.append(len(a['array']) / a['sampling_rate'])

In [24]:
durations = np.array(durations)

In [25]:
f"min: {durations.min():.1f}s, max: {durations.max():.1f}s"

'min: 0.7s, max: 29.3s'

In [26]:
,f"mean: {durations.mean():.1f}s, median: {np.median(durations):.1f}s"

'mean: 10.7s, median: 8.4s'

In [37]:
import json
from collections import Counter

In [38]:
all_text = " ".join([s['transcription'].lower() for s in train_subset])
chars = Counter(all_text)

In [39]:
vocab_dict = {c: i for i, c in enumerate(sorted(set(all_text))) if c.isalpha() or c == " "}

In [40]:
vocab_dict['[UNK]'] = len(vocab_dict)
vocab_dict['[PAD]'] = len(vocab_dict)
vocab_dict['|'] = vocab_dict.pop(' ')

In [44]:
print(f"vocab size: {len(vocab_dict)}")
print(f"chars: {''.join([c for c in vocab_dict.keys() if c not in ['[UNK]', '[PAD]', '|']])}")

vocab size: 39
chars: abcdefghijklmnopqrstuvwxyzâçéîöûüğış


In [45]:
with open('/content/vocab.json', 'w', encoding='utf-8') as f:
    json.dump(vocab_dict, f, ensure_ascii=False)

In [46]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC

In [47]:
tokenizer = Wav2Vec2CTCTokenizer(
    '/content/vocab.json',
    unk_token='[UNK]',
    pad_token='[PAD]',
    word_delimiter_token='|'
)

In [48]:
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0,
    do_normalize=True, return_attention_mask=True
)

In [49]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [50]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    layerdrop=0.05,
    ctc_loss_reduction="mean",
    pad_token_id=tokenizer.pad_token_id,
    vocab_size=len(tokenizer)
)

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/422 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Wav2Vec2ForCTC LOAD REPORT from: facebook/wav2vec2-large-xlsr-53
Key                          | Status     | 
-----------------------------+------------+-
quantizer.codevectors        | UNEXPECTED | 
project_hid.weight           | UNEXPECTED | 
project_q.bias               | UNEXPECTED | 
quantizer.weight_proj.weight | UNEXPECTED | 
project_hid.bias             | UNEXPECTED | 
quantizer.weight_proj.bias   | UNEXPECTED | 
project_q.weight             | UNEXPECTED | 
lm_head.weight               | MISSING    | 
lm_head.bias                 | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [51]:
model.freeze_feature_encoder()

In [52]:
print(f"model params: {model.num_parameters() / 1e6:.0f}M")

model params: 315M


In [53]:
def preprocess(batch):
    audio = batch['audio']

    inputs = processor(
        audio['array'],
        sampling_rate=audio['sampling_rate'],
        return_tensors=None
    )
    batch['input_values'] = inputs.input_values[0]
    batch['input_length'] = len(batch['input_values'])

    text = batch['transcription'].lower().strip()
    batch['labels'] = processor.tokenizer(text).input_ids

    return batch

In [54]:
train_processed = train_subset.map(preprocess, remove_columns=train_subset.column_names)
test_processed = test_subset.map(preprocess, remove_columns=test_subset.column_names)

Map:   0%|          | 0/3861 [00:00<?, ? examples/s]

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

In [55]:
max_len = 20 * 16000 #20 saniyədən çoxları atırıq ki crash falan verməsin
train_processed = train_processed.filter(lambda x: x['input_length'] < max_len)

Filter:   0%|          | 0/3861 [00:00<?, ? examples/s]

In [56]:
print(f"train: {len(train_processed)}, test: {len(test_processed)}")

train: 3310, test: 203


In [57]:
import torch
from dataclasses import dataclass
from typing import List, Dict, Union
from jiwer import wer

In [58]:
@dataclass
class CTCCollator:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict]) -> Dict[str, torch.Tensor]:
        input_vals = [{"input_values": f["input_values"]} for f in features]
        label_vals = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.pad(input_vals, padding=self.padding, return_tensors="pt")

        labels_batch = self.processor.tokenizer.pad(label_vals, padding=self.padding, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels

        return batch


In [59]:
collator = CTCCollator(processor=processor)

In [61]:

def compute_metrics(pred):
    logits = pred.predictions
    pred_ids = np.argmax(logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id # -100 decode üçün

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    w = wer(label_str, pred_str)
    return {"wer": w}

In [62]:
from transformers import TrainingArguments, Trainer

In [63]:
training_args = TrainingArguments(
    output_dir="/content/wav2vec2-tr",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=3e-4,
    warmup_steps=300,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    report_to="tensorboard",
    logging_dir="/content/tb_logs",
    save_total_limit=2,
    dataloader_num_workers=2,
    group_by_length=True,  # ramın ekonomiyası üçün
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [67]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    layerdrop=0.05,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer) + 2) # BOS/EOS görə, xəta vermişdi

Loading weights:   0%|          | 0/422 [00:00<?, ?it/s]

Wav2Vec2ForCTC LOAD REPORT from: facebook/wav2vec2-large-xlsr-53
Key                          | Status     | 
-----------------------------+------------+-
quantizer.codevectors        | UNEXPECTED | 
project_hid.weight           | UNEXPECTED | 
project_q.bias               | UNEXPECTED | 
quantizer.weight_proj.weight | UNEXPECTED | 
project_hid.bias             | UNEXPECTED | 
quantizer.weight_proj.bias   | UNEXPECTED | 
project_q.weight             | UNEXPECTED | 
lm_head.weight               | MISSING    | 
lm_head.bias                 | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [68]:
model.freeze_feature_encoder()

In [71]:
max_label = max(max(x['labels']) for x in train_processed)
print(f"max label id: {max_label}")
print(f"tokenizer vocab size: {len(processor.tokenizer)}")
print(f"vocab: {processor.tokenizer.get_vocab()}")

max label id: 65
tokenizer vocab size: 41
vocab: {'a': 29, 'b': 30, 'c': 31, 'd': 32, 'e': 33, 'f': 34, 'g': 35, 'h': 36, 'i': 37, 'j': 38, 'k': 39, 'l': 40, 'm': 41, 'n': 42, 'o': 43, 'p': 44, 'q': 45, 'r': 46, 's': 47, 't': 48, 'u': 49, 'v': 50, 'w': 51, 'x': 52, 'y': 53, 'z': 54, 'â': 56, 'ç': 57, 'é': 58, 'î': 59, 'ö': 60, 'û': 61, 'ü': 62, 'ğ': 63, 'ı': 64, 'ş': 65, '[UNK]': 37, '[PAD]': 38, '|': 0, '<s>': 39, '</s>': 40}




Vocabulary düzgün yaranmamışdı, simvolları 0dan yığıram yenə


In [72]:
all_text = " ".join([s['transcription'].lower() for s in train_subset])
chars = sorted(set(c for c in all_text if c.isalpha() or c == " "))

In [74]:
vocab_dict = {"[PAD]": 0, "[UNK]": 1, "|": 2}
for i, c in enumerate(chars):
    if c == " ":
        continue
    vocab_dict[c] = i + 3

print(f"vocab size: {len(vocab_dict)}")
print(vocab_dict)

with open('/content/vocab.json', 'w', encoding='utf-8') as f:
    json.dump(vocab_dict, f, ensure_ascii=False)

vocab size: 39
{'[PAD]': 0, '[UNK]': 1, '|': 2, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'j': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'q': 20, 'r': 21, 's': 22, 't': 23, 'u': 24, 'v': 25, 'w': 26, 'x': 27, 'y': 28, 'z': 29, 'â': 30, 'ç': 31, 'é': 32, 'î': 33, 'ö': 34, 'û': 35, 'ü': 36, 'ğ': 37, 'ı': 38, 'ş': 39}


In [75]:
tokenizer = Wav2Vec2CTCTokenizer(
    '/content/vocab.json',
    unk_token='[UNK]',
    pad_token='[PAD]',
    word_delimiter_token='|'
)

In [76]:
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0,
    do_normalize=True, return_attention_mask=True
)

In [77]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [78]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    layerdrop=0.05,
    ctc_loss_reduction="mean",
    pad_token_id=tokenizer.pad_token_id,
    vocab_size=len(tokenizer)
)

Loading weights:   0%|          | 0/422 [00:00<?, ?it/s]

Wav2Vec2ForCTC LOAD REPORT from: facebook/wav2vec2-large-xlsr-53
Key                          | Status     | 
-----------------------------+------------+-
quantizer.codevectors        | UNEXPECTED | 
project_hid.weight           | UNEXPECTED | 
project_q.bias               | UNEXPECTED | 
quantizer.weight_proj.weight | UNEXPECTED | 
project_hid.bias             | UNEXPECTED | 
quantizer.weight_proj.bias   | UNEXPECTED | 
project_q.weight             | UNEXPECTED | 
lm_head.weight               | MISSING    | 
lm_head.bias                 | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [79]:
model.freeze_feature_encoder()

In [80]:
print(f"vocab size: {len(tokenizer)}, model vocab: {model.config.vocab_size}")

vocab size: 41, model vocab: 41


yeni tokenizerlə bir də yoxlayırıq

In [81]:
train_processed = train_subset.map(preprocess, remove_columns=train_subset.column_names)
test_processed = test_subset.map(preprocess, remove_columns=test_subset.column_names)

Map:   0%|          | 0/3861 [00:00<?, ? examples/s]

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

In [82]:
max_len = 20 * 16000 #atırıq 20 sdən çoxları ekonomiya üçün
train_processed = train_processed.filter(lambda x: x['input_length'] < max_len)

Filter:   0%|          | 0/3861 [00:00<?, ? examples/s]

In [83]:
max_label = max(max(x['labels']) for x in train_processed)
print(f"max label: {max_label}, vocab size: {len(tokenizer)}")
assert max_label < len(tokenizer), "LABEL MISMATCH!"
print("all good")

max label: 39, vocab size: 41
all good


In [84]:
collator = CTCCollator(processor=processor)

In [85]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_processed,
    eval_dataset=test_processed,
    data_collator=collator,
    compute_metrics=compute_metrics,
    processing_class=processor,
)


In [86]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 40, 'bos_token_id': 39}.


Epoch,Training Loss,Validation Loss,Wer
1,12.403690,2.939410,0.997912
2,3.377078,0.564843,0.561326
3,2.309772,0.420767,0.416232
4,1.981728,0.381596,0.355167
5,1.692940,0.366459,0.340292


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=1035, training_loss=6.3406786084751, metrics={'train_runtime': 2420.1492, 'train_samples_per_second': 6.838, 'train_steps_per_second': 0.428, 'total_flos': 4.0831322277519104e+18, 'train_loss': 6.3406786084751, 'epoch': 5.0})

In [87]:
model_path = "/content/wav2vec2-tr-final"

In [88]:
trainer.save_model(model_path)

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

In [89]:
processor.save_pretrained(model_path)

['/content/wav2vec2-tr-final/processor_config.json']

In [90]:
sample = test_subset[0]
inputs = processor(sample['audio']['array'], sampling_rate=16000, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs.to(model.device)).logits
pred_ids = torch.argmax(logits, dim=-1)
pred_text = processor.batch_decode(pred_ids)[0]

In [91]:
print(f"reference: {sample['transcription'].lower()}")
print(f"predicted: {pred_text}")

reference: yerin yakınında yer çekimi ivmesi 9,8 metre bölü saniye kare ve aşağı yönlüdür.
predicted: yerin yakında yer çekimeinmesi  metrebölü saniye kare ve aşağı yönlüdür aşağı dordur[UNK]


In [92]:
import shutil

In [93]:
shutil.make_archive("/content/wav2vec2-tr-final", 'zip', model_path)

'/content/wav2vec2-tr-final.zip'

In [94]:
from google.colab import files

In [95]:
files.download("/content/wav2vec2-tr-final.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>